Script used to generate pseudo labels for unlabelled data.

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import Dataset

from dpt import DepthSense

In [3]:
torch.random.manual_seed(7643)

In [8]:
# Parameters and hyperparameters used for training.
description: str = "DepthSense for Metric Depth and Normal Estimation"
model_path: str = "models/teacher_{}.pth"

batch_size: int = 1
dataset_name: Dataset = "hypersim"
device: str = "cpu"
encoder: str = "vits"
eps: float = 1e-8
features: int = 128

dataset_path = Path(f"datasets/pseudo/{dataset_name}")

# Model initialization.
model_name: str = model_path.replace("{}", dataset_name)
model: DepthSense = DepthSense(encoder, features)
#model.load_state_dict(torch.load(model_name, map_location=device, weights_only=True))

In [9]:
# Pseudo-labeling.
model.eval()

running_loss: float = 0.0
for i, file in enumerate(dataset_path.iterdir()):
    if file.is_dir():
        # Move to appropriate device.
        x = (
            torch.from_numpy(np.load(file / "frame.npy"))
            .to(device)
            .unsqueeze(0)
            .permute(0, 3, 1, 2)
            .float()
        )
        print(x.shape)
        # Forward pass.
        z, n = model.infer_image(x)
        np.save(file / "depth.npy", np.array(z))
        np.save(file / "normal.npy", np.array(n))

torch.Size([1, 3, 768, 1024])
